# Исследование решения задачи распознавания именованных сущностей (NER)

# 1. Постановка задачи

Одной из основных причин, по которой сотрудники первой линии поддержки вынуждены затрачивать дополнительное время на вспомогательную коммуникацию с клиентом, является отсутствие серийного номера оборудования в тексте письма, которое присылается самим клиентом по электронной почте, поскольку по серийному номеру оператор может узнать необходимую информацию об оборудовании.

Для того, чтобы минимизировать затраченное время сотрудника первой линии поддержки на обработку обращения, разрабатываемая система должна решать задачу распознавания серийных номеров в тексте пользовательского письма.

Благодаря этому система может зафиксировать факт недостаточности информации в письме (отсутствие полноты) и не пропускать сообщение дальше до ее уточнения. И уже после выделения всей необходимой информации только передавать оператору текст запроса.

# 2. Формирование окружения

Для того, чтобы решить данную задачу, импортируем необходимые библиотеки (open source) для работы, а также укажем, какую роль данные библиотеки играют в ходе исследования:
- **pandas**: используется для представления исходных данных в виде объекта таблицы, который инкапсулирует методы работы со строками, столбцами и генерируемыми выборками;
- **numpy**: используется для работы с данными, представленными в виде массивов и матриц;
- **typing**: модуль Python, необходимый для типизации аргументов функций.

Спецификация версий модулей и библиотек указана в README.md.

In [283]:
import pandas as pd
import numpy as np
import typing as t

# 3. Загрузка и обработка данных

Для того, чтобы приступить к решению задачи, необходимо загрузить данные в память, а также предобработать их, чтобы они были представлены без аномалий, которые могут пагубно повлиять на анализ и конечный результат.

## 3.1. Представление данных в Python

Для чтения данных будем использовать библиотеку pandas, так как сами данные представлены в формате .csv (разделитель -- запятая), а pandas имеет простой в использовании интерфейс для загрузки такого рода данных с помощью метода read_csv(...), который принимает:
- путь до файла с данными;
- кодировку;
- разделитель в .csv файле (в данном случае он не указывается, так как, по умолчанию, запятая является базовым разделителем).

Воспользуемся методом и загрузим данные.

In [209]:
data = pd.read_csv("../data/train_data.csv", encoding="utf-8")
data

,index,Тема,Описание,Тип оборудования,Точка отказа,Серийный номер
0,0,Не работает блок питания,В течении недели перестал внезапно работать бл...,Ноутбук,Блок питания,C223100360
1,1,Не включается SILA LLC HK2-3404.,Добрый день!_x000D_\r\nПодскажите пожалуйста ч...,Ноутбук,Материнская плата,C223094534
2,2,Сервер СР2-5422 // Системные ошибки // D251110041,Добрый вечер! На сервере появились системные о...,Сервер,Материнская плата,D251110041
3,3,Повреждение матрицы,Прошу произвести диагностику и сориентировать ...,Ноутбук,Матрица,C223014328
4,4,sila HK2-1404 c223014125 asset 7101087,Добрый день!\r\nПрошу взять ноутбук sila HK2-1...,Ноутбук,Материнская плата,C223014125
...,...,...,...,...,...,...
189,189,НК2-1404 // Не работает Wi-FI (Кашира)// C2231...,На устройстве не работает WiFi\r\n,Ноутбук,Wi-fi модуль,C223101219
190,190,Не работает блок питания (зарядка),Не работает блок питания (зарядка),Ноутбук,Блок питания,С223093223
191,191,Ноутбук НК2-1404 //Не работает клавиша F1 или ...,"Добрый день,\r\nпосле ремонта ноутбука переста...",Ноутбук,Клавиатура,С223010731
192,192,Отказ блока питания,Добрый день! \r\n У нас перестал работать заря...,Ноутбук,Блок питания,C222091750


## 3.2. Обработка Описания

В исходном наборе данных в столбце "Описание" присутствовали аномальные символы, возникшие из-за дефекта кодировки Unicode: _x000D_. Данные символы необходимо удалить, поскольку они не определяются, как часть слова. И не являются лексемами.

Удаление будем осуществлять средствами pandas. Для этого представим столбец с описанием как строку при помощи метода .str, после чего при помощи строкового метода replace явно заменим каждое вхождение '_x000D_' на пустой символ.

В результате получим датафрейм, в котором все значения являются лексически достоверными, бех специальных символов. 

In [210]:
data['Описание'] = data['Описание'].str.replace('_x000D_', '', regex=False)
data

,index,Тема,Описание,Тип оборудования,Точка отказа,Серийный номер
0,0,Не работает блок питания,В течении недели перестал внезапно работать бл...,Ноутбук,Блок питания,C223100360
1,1,Не включается SILA LLC HK2-3404.,Добрый день!\r\nПодскажите пожалуйста что може...,Ноутбук,Материнская плата,C223094534
2,2,Сервер СР2-5422 // Системные ошибки // D251110041,Добрый вечер! На сервере появились системные о...,Сервер,Материнская плата,D251110041
3,3,Повреждение матрицы,Прошу произвести диагностику и сориентировать ...,Ноутбук,Матрица,C223014328
4,4,sila HK2-1404 c223014125 asset 7101087,Добрый день!\r\nПрошу взять ноутбук sila HK2-1...,Ноутбук,Материнская плата,C223014125
...,...,...,...,...,...,...
189,189,НК2-1404 // Не работает Wi-FI (Кашира)// C2231...,На устройстве не работает WiFi\r\n,Ноутбук,Wi-fi модуль,C223101219
190,190,Не работает блок питания (зарядка),Не работает блок питания (зарядка),Ноутбук,Блок питания,С223093223
191,191,Ноутбук НК2-1404 //Не работает клавиша F1 или ...,"Добрый день,\r\nпосле ремонта ноутбука переста...",Ноутбук,Клавиатура,С223010731
192,192,Отказ блока питания,Добрый день! \r\n У нас перестал работать заря...,Ноутбук,Блок питания,C222091750


## 3.3. Обработка Серийного номера

Также поработаем с целевым столбцом, который содержит информацию о серийных номерах в конкретном обращении, для их дальнейшего исследования. 

Для того, чтобы проанализировать данные в столбце, необходимо выделить переменную, содержащую значения столбца. Для этого в переменную X поместим значение столбца, обращаясь к нему по наименованию колонки. Отобразим результат.

In [213]:
X = data['Серийный номер']
X

0      C223100360
1      C223094534
2      D251110041
3      C223014328
4      C223014125
          ...    
189    C223101219
190    С223093223
191    С223010731
192    C222091750
193    D263080001
Name: Серийный номер, Length: 194, dtype: object

Затем приведем переменную типа данных pd.Series к List при помощи соответствующего метода. Сделано это для того, чтобы можно было пользоваться функцией map из Python в будущем.

In [214]:
X_list = X.to_list()
X_list

['C223100360',
 'C223094534',
 'D251110041',
 'C223014328',
 'C223014125',
 'C223010123',
 'C223091001',
 'C223012430',
 'D252030021',
 'c223012961',
 'C223100312',
 'C223090725',
 'C223102255',
 'CKM01212505744',
 'С111120102',
 'D252110012',
 'С112040045',
 'C222090950',
 'C223090096',
 'C223010035',
 'C223102298',
 'C223091126',
 'C223010076',
 'C223013256',
 'С223090320',
 'C223013523',
 'C223014561',
 'C223010345, С223092735',
 'D263120004',
 'D114030017(FTKY614)',
 'С111080061',
 'C223010446',
 'D252030009',
 'C223011170, C223102198, C223011889, C223101201, C223100299, C223011531',
 'D251110015',
 'C222090246',
 'D254020003',
 'C223091623',
 'C223012998 C223101389  C223102298   C223100876  C223100050',
 'D263090003',
 'C223013250',
 'C223011660',
 'C233041042',
 'C223011088',
 'С111080030',
 'C223091624',
 'D251110011',
 'C223014954',
 'С223010395',
 'C223012095',
 'С223091324',
 'С223091153',
 'C223090893',
 'C223090123',
 'C223010423, C223011174',
 'C223011046',
 'C223011355',


Получили список, содержащий для каждого объекта его целевые номера, записанные в виде строки.

# 4. Выделение паттернов Серийного номера

Для того, чтобы построить алгоритм распознавания Серийных номеров, необходимо выделить всевозможные паттерны того, как эти Серийные номера выглядят. Для этого проанализируем их структуру.

## 4.1. Разбиение объектов строки

В первоначальных данных может возникнуть ситуация, что в одном письме от пользователя описаны несколько серийных номеров. И в датасете это перечисление может записываться одним из следующих образов:
1. через запятую;
2. через пробел;
3. через символ //.

При всем этом по типу данных такая запись является единой строкой, поэтому необходимо унифицировать запись разделения, а также распарсить строку на отдельные серийные номера.

При помощи регулярных выражений построим цепочку преобразований, которая сводит все вышеперечисленные случаи к случаю, когда данные разделены при помощи пробела.

После того, как произвели унификацию регулярными выражениями, вызовем метод .split() у получившейся строки, которая разделит ее на список Серийных номеров, разделенных пробелом.

Данные преобразования выполним для каждого элемента, поэтому оформим их в виде lambda-выражения, которое применяется к каждому элементу столбца с признаками.

In [215]:
import re


list_ids = list(map(lambda s: re.sub(', ', ' ', re.sub(' // ', ' ', s)).split(), X_list))
list_ids

[['C223100360'],
 ['C223094534'],
 ['D251110041'],
 ['C223014328'],
 ['C223014125'],
 ['C223010123'],
 ['C223091001'],
 ['C223012430'],
 ['D252030021'],
 ['c223012961'],
 ['C223100312'],
 ['C223090725'],
 ['C223102255'],
 ['CKM01212505744'],
 ['С111120102'],
 ['D252110012'],
 ['С112040045'],
 ['C222090950'],
 ['C223090096'],
 ['C223010035'],
 ['C223102298'],
 ['C223091126'],
 ['C223010076'],
 ['C223013256'],
 ['С223090320'],
 ['C223013523'],
 ['C223014561'],
 ['C223010345', 'С223092735'],
 ['D263120004'],
 ['D114030017(FTKY614)'],
 ['С111080061'],
 ['C223010446'],
 ['D252030009'],
 ['C223011170',
  'C223102198',
  'C223011889',
  'C223101201',
  'C223100299',
  'C223011531'],
 ['D251110015'],
 ['C222090246'],
 ['D254020003'],
 ['C223091623'],
 ['C223012998', 'C223101389', 'C223102298', 'C223100876', 'C223100050'],
 ['D263090003'],
 ['C223013250'],
 ['C223011660'],
 ['C233041042'],
 ['C223011088'],
 ['С111080030'],
 ['C223091624'],
 ['D251110011'],
 ['C223014954'],
 ['С223010395'],
 ['C

В итоге получаем список списков Серийных номеров для каждого обращения, готовый для дальнейшего анализа. ОДнако по типу данных такая запись будет являться двумерным массивом, то есть в одной ячейке переменной list_ids находится список, который содержит больше одного значения Серийного номера.

Для дальнейшего исследования необходимо спрямить полученный набор данных до одномерного массива. Выполним это при помощи модуля Python itertools, а также метода chain, который представляет итерационный объект в виде одного потока данных.

Полученный объект приведем к листу, чтобы можно было обращаться к элементам по индексу

In [216]:
import itertools


flat_list = list(itertools.chain(*list_ids))
flat_list

['C223100360',
 'C223094534',
 'D251110041',
 'C223014328',
 'C223014125',
 'C223010123',
 'C223091001',
 'C223012430',
 'D252030021',
 'c223012961',
 'C223100312',
 'C223090725',
 'C223102255',
 'CKM01212505744',
 'С111120102',
 'D252110012',
 'С112040045',
 'C222090950',
 'C223090096',
 'C223010035',
 'C223102298',
 'C223091126',
 'C223010076',
 'C223013256',
 'С223090320',
 'C223013523',
 'C223014561',
 'C223010345',
 'С223092735',
 'D263120004',
 'D114030017(FTKY614)',
 'С111080061',
 'C223010446',
 'D252030009',
 'C223011170',
 'C223102198',
 'C223011889',
 'C223101201',
 'C223100299',
 'C223011531',
 'D251110015',
 'C222090246',
 'D254020003',
 'C223091623',
 'C223012998',
 'C223101389',
 'C223102298',
 'C223100876',
 'C223100050',
 'D263090003',
 'C223013250',
 'C223011660',
 'C233041042',
 'C223011088',
 'С111080030',
 'C223091624',
 'D251110011',
 'C223014954',
 'С223010395',
 'C223012095',
 'С223091324',
 'С223091153',
 'C223090893',
 'C223090123',
 'C223010423',
 'C223011174

Получаем единую структуру, которая содержит всевозможные Серийные номера, содержащиеся в наборе данных.

Размер такого ряда: 231 элемент.

In [217]:
len(flat_list)

231

## 4.2. Анализ наличия паттернов в Серийном номере

После того, как у нас появился ряд серийных номеров, попробуем провести их кластеризацию и выделить паттерны того, как выглядит Серийный номер, в целом.

Сперва приведем сформированный ранее массив элементов к массиву из библиотеки numpy, чтобы проще и удобнее в дальнейшем обращаться к элементам массива и его срезам.

In [218]:
flat_list = np.array(flat_list)
flat_list

array(['C223100360', 'C223094534', 'D251110041', 'C223014328',
       'C223014125', 'C223010123', 'C223091001', 'C223012430',
       'D252030021', 'c223012961', 'C223100312', 'C223090725',
       'C223102255', 'CKM01212505744', 'С111120102', 'D252110012',
       'С112040045', 'C222090950', 'C223090096', 'C223010035',
       'C223102298', 'C223091126', 'C223010076', 'C223013256',
       'С223090320', 'C223013523', 'C223014561', 'C223010345',
       'С223092735', 'D263120004', 'D114030017(FTKY614)', 'С111080061',
       'C223010446', 'D252030009', 'C223011170', 'C223102198',
       'C223011889', 'C223101201', 'C223100299', 'C223011531',
       'D251110015', 'C222090246', 'D254020003', 'C223091623',
       'C223012998', 'C223101389', 'C223102298', 'C223100876',
       'C223100050', 'D263090003', 'C223013250', 'C223011660',
       'C233041042', 'C223011088', 'С111080030', 'C223091624',
       'D251110011', 'C223014954', 'С223010395', 'C223012095',
       'С223091324', 'С223091153', 'C22309

Сформируем 3 объекта: 
1. список length_list, состоящий из длины каждого Серийного номера (то есть для каждого элемента flat_list, сформированного ранее, посчитаем длину строки); 
2. список val, содержащий уникальные значения длин Серийного номера;
3. список counts, подсчитывающий количество вхождений каждой уникальной длины в список flat_list.

данные объекты сформируем при помощи средств Python и numpy.

In [221]:
length_list = np.array(list(map(lambda s: len(s), flat_list)))
val, counts = np.unique(length_list, return_counts=True)
val, counts

(array([10, 14, 19]), array([219,  11,   1]))

Данные объекты необходимы для того, чтобы провести кластеризацию по длине Серийного номера. С дальнейшей проверки гипотезы о том, что структура у сгруппированных по длине элементов будет присутствовать однозначная.

Как видно, удалось выделить 3 группы:
1. Серийный номер, состоящий из 10 символов (219 случаев);
2. Серийный номер, состоящий из 14 символов (11 случаев);
3. Серийный номер, состоящий из 19 символов (1 случай).

Сформируем списки, в которых будут находится исключительно серийные номера, подпадающие под один из трех вышеуказанных случаев.

In [230]:
flat_list_10 = flat_list[length_list == 10]
flat_list_14 = flat_list[length_list == 14]
flat_list_19 = flat_list[length_list == 19]

### 4.2.1. Анализ первого кластера

Проведем анализ первого кластера, который состоит из серийных номеров длины 10.

Рассмотрим случайного представителя данного кластера.

In [238]:
flat_list_10[0].item()

'C223100360'

Видим, что он состоит из буквенных символов и цифр, расположенных на определенной позиции. 

Проверим гипотезу, что все серийные номера длины 10 строятся по следующему правилу: сначала идет роно одна буква (регистр не важен), а затем идет 9 произвольных цифр.

Для проверки гипотезы построим вспомогательную конструкцию. Каждому элементу из данного кластера поставим в соответствие булевый флаг True в том случае, если данные объект принадлежит вышеуказанной структуре, а также False -- в обратном. Реализуем это при помощи лямбда функции, использующей метод isalpha() для необходимой подстроки, а также функции map.

Будем считать, что гипотеза принята, если это свойство выполняется для КАЖДОГО элемента кластера. Установим это при помощи функции Python all, которая пример в аргумент массив индикаторов. Функция вернет true только в том случае, если все элементы будут удовлетворять гипотезе.

In [232]:
all(list(map(lambda s: s[0].isalpha() and not s[1:].isalpha(), flat_list_10)))

True

В результате получаем, что гипотеза подтверждается! Один из паттернов поведения Серийного номера выглядит следующим образом (в терминах регулярных выражений): \s\d{9}. 

### 4.2.2. Анализ второго кластера

Также рассмотрим случайного представителя второго кластера. 

In [239]:
flat_list_14[0].item()

'CKM01212505744'

Аналогично предыдущему пункту, проверим гипотезу о том, что серийные номера длины 14 строятся следующим образом: сперва идет 3 буквенных символа (регистр не важен), после этого идут 11 численных значений.

In [240]:
all(list(map(lambda s: s[:3].isalpha() and not s[3:].isalpha(), flat_list_14)))

True

Получаем, что гипотеза принимается! Второму кластеру соответствует следующая запись в терминах регулярных выражений: \s{3}\d{11}.

### 4.2.3. Анализ третьего кластера

Рассмотрим единственного представителя третьего кластера, а также его структуру.

In [286]:
flat_list_19[0].item()

'D114030017(FTKY614)'

Видим, что данный элемент состоит из двух элементов: последовательности до скобки и скобки с последовательностью внутри.

Важно отметить, что последовательность до скобки соответствует полностью первому кластеру. 

В ходе экспертной сессии было установлено, что существуют Серийные номера, имеющие скобки. Это справедливо как для первого кластера, так и для второго, то есть каждый кластер имеет субкластер, состоящий из его представителя, а также скобок. Поэтому сформируем два общих паттерна в терминах регулярных выражений, которые справедливы для каждого кластера и предполагают наличие скобок:
1. \b[A-Za-zА-Яа-я]\d{9}\b(?:\([^\)]*\))?,
2. \b[A-Za-zА-Яа-я]{3}\d{11}\b(?:\([^\)]*\))?.

Конструкция [A-Za-zА-Яа-я] и [A-Za-zА-Яа-я]{3} отвечает за буквенный символ, а также число его вхождений.

Конструкция \d{$\ i\ $} отвечает за размер числовой последовательности в шаблоне (длины $i \in \mathbb{N}$).

Повторяющаяся конструкция (?:\([^\)]*\))? как раз отвечает за опциональное наличие скобок.

# 5. Формирование функционала для поиска именованных сущностей

После того, как были выделены кластеры в данных, реализуем функционал для выделения Серийных номеров из данных.

Логику по выделению Серийных номеров будем реализовывать при помощи регулярных выражений.

У нас есть четко фиксированная структура Серийного номера, есть два строгих паттерна, поэтому, по сравнению с нейросетевыми методами выделения именованных сущностей, подход регулярных выражений имеет ряд преимуществ:
1. точность: регулярные выражения не представляют из себя структуру "черного ящика", это алгоритмический подход с абсолютной точностью;
2. скорость работы: в отличие от алгоритмов машинного обучения, поиск всех вхождений при помощи регулярного выражения обладает алгоритмической сложностью $\Omicron(n)$, то есть линейной, в то время как алгоритмы ML, решающие данную задачу, стартуют с квадратичной сложности;
3. независимость от контекста и пользовательского ввода: данный подход не зависит от того, как именно пользователь вводит информацию, она исключительно ищет факт наличия информации, поэтому отказоусойчивость очень высокая;
4. независимость от тренировочного датасета: подход не требует данных для обучения, поэтому рабоатет из коробки, его не нужно дообучать, он не подвержен переобучению или недообучению, не зависит от размера выборки (метод НС может просто не понять зависимости из-за дисбаланса классов токенов по причине небольшого количества обучающих примеров);
5. устойчивость к регистру символов и схожим буквам в разных алфавитах: для регулярных выражений не имеет разницы формат строки.

В то же время методы регулярных выражений и НС не устойчивы к появлению новых кластеров серийных номеров, однако при увеличении обучающего датасета можно рассмотреть подход zero-shot NER с использованием LLM, дообученных для задачи NER. 

В данных условиях наиболее оптимальным является метод регулярных выражений. Поэтому имплементируем логику по выделению именованной сущности Серийный номер.

## 5.1. Имплементация основных функций

### 5.1.1. Логика по предобработке результирующих значений

Предусмотрим возможность преобразовывать найденные серийные номера к стандартизированному виду. 

Поскольку серийные номера могут находиться в произвольном регистре, добавим возможность унифицировать результаты.

Также в данных присутствует неоднозначность с буквенными символами: может возникнуть ситуация, что пользователь пишет Серийный номер русскими буквами, а не английскими.

Поэтому реализуем функционал по предобработке Серийных номеров. 

Имплементируем функцию, которая переводит русские символы, схожие по начертанию, в английские. Для этого воспользуемся методом класса str maketrans, которая строит таблицу соответствия символов, а затем при помощи метода translate меняет символы по соответствию.

In [347]:
def russian_to_english(text):
    """переводит русские символы в английские, схожие по начертанию

    Args:
        text (_type_): Входной текст для преобразования

    Returns:
        _type_: Результат преобразования
    """
    russian = "СМТВАРОКЕНХ"
    english = "CMTBAPOKEHX"
    translation_table = str.maketrans(russian, english)
    return text.translate(translation_table)

Протестируем работу метода на тестовой строке, которая содержит русский символ. Вывод ячейки будет True, если преобразование прошло успешно.

In [306]:
russian_text = "М222090950"
english_text = russian_to_english(russian_text)
print(english_text == 'M222090950')

True


Также имплементируем итоговую функцию, которая помимо перевода символов меняет регистр.

In [348]:
def id_preprocessing(id: str) -> str:
    """Переводит Серийный номер в канонический вид (Английские буквы + верхний регистр)

    Args:
        id (str): Исходный серийный номер

    Returns:
        str: Серийный номер в каноническом виде
    """
    return russian_to_english(id.upper())

### 5.1.2. Логика по выделению Серийного номера

Необходимо реализовать итоговый набор функций по выделению первого серийного номера, а также всех серийных номеров в тексте письма.

Занесем паттерны каждого кластера Серийного номера в отдельные переменные.

In [303]:
patterns = [
    r'\b[A-Za-zА-Яа-я]\d{9}\b(?:\([^\)]*\))?',
    r'\b[A-Za-zА-Яа-я]{3}\d{11}\b(?:\([^\)]*\))?',
]

Имплементируем функцию, которая возвращает все Серийные номера, встреченные в заголовке и теле письма.

In [342]:
def get_all_serial_numbers(header: str, body: str, patterns: t.List[str], preprocessing: t.Callable[[str], str] = lambda s: s, default_value: str = 'Уточнить') -> t.List[str]:
    """Возвращает все найденные Серийные номера в тексте заголовка письма и его тела

    Args:
        header (str): Заголовок письма
        body (str): Тело письма
        patterns (t.List[str]): Признаки кластеров Серийных номеров
        preprocessing (_type_, optional): Функция по предобработке результирующих Серийных номеров. Defaults to lambdas:s.

    Returns:
        t.List[str]: Список всех найденных Серийных номеров в письме
    """
    res = list(
        map(
            lambda s: preprocessing(s), 
            itertools.chain(*[
                set(re.findall(pattern, header) + re.findall(pattern, body))
                    for pattern in patterns
            ])
        )
    )
    if res:
        return res
    return [default_value]

Протестируем данную функцию на случайной строке из датасета.

In [315]:
row = data.iloc[54]

header = row['Тема']
body = row['Описание']

res = row['Серийный номер']

In [310]:
header

'10552. СИЛА HK-1404 // замена блоков питания // C223010423, C223011174'

In [311]:
body

'Добрый день! Не работают блоки питания, прошу заменить\r\n'

In [312]:
res

'C223010423, C223011174'

In [ ]:
get_all_serial_numbers(header, body, patterns)

['C223011174', 'C223010423']

Имплементируем функцию по поиску первого вхождения Серийного номера.

In [346]:
def get_one_serial_number(header: str, body: str, patterns: t.List[str], preprocessing: t.Callable[[str], str] = lambda s: s, default_value: str = 'Уточнить') -> str:
    """Возвращает первое вхождение Серийного номера в тексте заголовка письма и его тела

    Args:
        header (str): Заголовок письма
        body (str): Тело письма
        patterns (t.List[str]): Признаки кластеров Серийных номеров
        preprocessing (_type_, optional): Функция по предобработке результирующих Серийных номеров. Defaults to lambdas:s.

    Returns:
        t.List[str]: Первое вхождение Серийного номера в письме
    """
    
    agg_pattern = '|'.join(patterns)
    res = re.findall(agg_pattern, ' '.join((header, body)))
    if res:
        return preprocessing(res[0])
    return default_value

И протестируем ее работу.

In [345]:
get_all_serial_numbers('Ноутбук НК2-1404', 'Не работает клавиша F1 или срабатывает с задержкой// С223010731(qw) в Й223010731(sas) D114030017(FTKY614) BSR12345678901(dfkj888)', patterns)

['Й223010731(sas)',
 'С223010731(qw)',
 'D114030017(FTKY614)',
 'BSR12345678901(dfkj888)']

# 6. Результирующий набор объектов

Перечислим в ячейках необходимые объекты и функции, необходимые для реализации API/

In [ ]:
def russian_to_english(text):
    """переводит русские символы в английские, схожие по начертанию

    Args:
        text (_type_): Входной текст для преобразования

    Returns:
        _type_: Результат преобразования
    """
    russian = "СМТВАРОКЕНХ"
    english = "CMTBAPOKEHX"
    translation_table = str.maketrans(russian, english)
    return text.translate(translation_table)

In [ ]:
def id_preprocessing(id: str) -> str:
    """Переводит Серийный номер в канонический вид (Английские буквы + верхний регистр)

    Args:
        id (str): Исходный серийный номер

    Returns:
        str: Серийный номер в каноническом виде
    """
    return russian_to_english(id.upper())

In [ ]:
patterns = [
    r'\b[A-Za-zА-Яа-я]\d{9}\b(?:\([^\)]*\))?',
    r'\b[A-Za-zА-Яа-я]{3}\d{11}\b(?:\([^\)]*\))?',
]

In [ ]:
def get_all_serial_numbers(header: str, body: str, patterns: t.List[str], preprocessing: t.Callable[[str], str] = lambda s: s, default_value: str = 'Уточнить') -> t.List[str]:
    """Возвращает все найденные Серийные номера в тексте заголовка письма и его тела

    Args:
        header (str): Заголовок письма
        body (str): Тело письма
        patterns (t.List[str]): Признаки кластеров Серийных номеров
        preprocessing (_type_, optional): Функция по предобработке результирующих Серийных номеров. Defaults to lambdas:s.

    Returns:
        t.List[str]: Список всех найденных Серийных номеров в письме
    """
    res = list(
        map(
            lambda s: preprocessing(s), 
            itertools.chain(*[
                set(re.findall(pattern, header) + re.findall(pattern, body))
                    for pattern in patterns
            ])
        )
    )
    if res:
        return res
    return [default_value]

In [ ]:
def get_one_serial_number(header: str, body: str, patterns: t.List[str], preprocessing: t.Callable[[str], str] = lambda s: s, default_value: str = 'Уточнить') -> str:
    """Возвращает первое вхождение Серийного номера в тексте заголовка письма и его тела

    Args:
        header (str): Заголовок письма
        body (str): Тело письма
        patterns (t.List[str]): Признаки кластеров Серийных номеров
        preprocessing (_type_, optional): Функция по предобработке результирующих Серийных номеров. Defaults to lambdas:s.

    Returns:
        t.List[str]: Первое вхождение Серийного номера в письме
    """
    
    agg_pattern = '|'.join(patterns)
    res = re.findall(agg_pattern, ' '.join((header, body)))
    if res:
        return preprocessing(res[0])
    return default_value